In [2]:
from keras import models,layers,optimizers
from keras.applications import VGG16

In [3]:
conv_base=VGG16(weights='imagenet',
               include_top=False,
               input_shape=(150,150,3))

In [4]:
model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [6]:
print('this is the num of trainable weight before freezing the conv base:',
     len(model.trainable_weights))

this is the num of trainable weight before freezing the conv base: 30


In [8]:
conv_base.trainable=False

In [9]:
print('this is num of trainable weight after freezing the conv base:',
     len(model.trainable_weights))

this is num of trainable weight after freezing the conv base: 4


In [10]:
from keras.preprocessing.image import ImageDataGenerator

In [13]:
import os
import numpy as np

In [14]:
base_dir='F:\\study\\ml\\DataSet\\dogs_vs_cats\\cats_and_dogs_small'
train_dir=os.path.join(base_dir,'train')
validation_dir=os.path.join(base_dir,'validation')
test_dir=os.path.join(base_dir,'test')

In [11]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest')

test_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                 target_size=(150,150),
                                                 batch_size=20,
                                                 class_mode='binary')

validation_generator=test_datagen.flow_from_directory(validation_dir,
                                                     target_size=(150,150),
                                                     batch_size=20,
                                                     class_mode='binary')

model.compile(loss='binary_crossentropy',
            optimizer=optimizers.RMSprop(lr=2e-5),
            metrics=['acc'])

In [19]:
history=model.fit_generator(train_generator,
                           steps_per_epoch=100,
                           epochs=30,
                           validation_data=validation_generator,
                           validation_steps=50)

Epoch 1/30
100/100 [==============================] - 378s 4s/step - loss: 0.6024 - acc: 0.6850 - val_loss: 0.4578 - val_acc: 0.8420
Epoch 2/30
100/100 [==============================] - 378s 4s/step - loss: 0.4890 - acc: 0.7850 - val_loss: 0.3822 - val_acc: 0.8570
Epoch 3/30
100/100 [==============================] - 376s 4s/step - loss: 0.4403 - acc: 0.8080 - val_loss: 0.3416 - val_acc: 0.8750
Epoch 4/30
100/100 [==============================] - 377s 4s/step - loss: 0.4089 - acc: 0.8205 - val_loss: 0.3499 - val_acc: 0.8570
Epoch 5/30
100/100 [==============================] - 377s 4s/step - loss: 0.3828 - acc: 0.8300 - val_loss: 0.2953 - val_acc: 0.8780
Epoch 6/30
100/100 [==============================] - 413s 4s/step - loss: 0.3741 - acc: 0.8415 - val_loss: 0.2901 - val_acc: 0.8830
Epoch 7/30
100/100 [==============================] - 390s 4s/step - loss: 0.3661 - acc: 0.8405 - val_loss: 0.2900 - val_acc: 0.8790
Epoch 8/30
100/100 [==============================] - 424s 4s/step - 

In [21]:
model.save('F:\study\ml\DataSet\dogs_vs_cats\cats_and_dogs_small\models\\cats_and_dogs_small_3.h5')

In [22]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [47]:
conv_base.trainable=True

In [48]:
print('this is num of trainable weight after freezing the conv base:',
     len(model.trainable_weights))

this is num of trainable weight after freezing the conv base: 10


In [42]:
# conv_base.trainable=False

In [53]:
for i in conv_base.layers:
    print(i.name)
    print(i.trainable)

input_1
False
block1_conv1
False
block1_conv2
False
block1_pool
False
block2_conv1
False
block2_conv2
False
block2_pool
False
block3_conv1
False
block3_conv2
False
block3_conv3
False
block3_pool
False
block4_conv1
False
block4_conv2
False
block4_conv3
False
block4_pool
False
block5_conv1
True
block5_conv2
True
block5_conv3
True
block5_pool
True


In [43]:
set_trainable=False
for layer in conv_base.layers:
    if layer.name=='block5_conv1':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False

In [50]:
# for i in conv_base.layers:
#     print(i.name)
#     print(i.trainable)

In [54]:
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-5),
             metrics=['acc'])

In [55]:
len(model.trainable_weights)

10

In [56]:
history=model.fit_generator(
train_generator,
steps_per_epoch=100,
epochs=20,
validation_data=validation_generator,
validation_steps=50)

Epoch 1/20
100/100 [==============================] - 453s 5s/step - loss: 0.2087 - acc: 0.9230 - val_loss: 0.2070 - val_acc: 0.9380
Epoch 2/20
100/100 [==============================] - 448s 4s/step - loss: 0.1751 - acc: 0.9335 - val_loss: 0.1893 - val_acc: 0.9380
Epoch 3/20
100/100 [==============================] - 445s 4s/step - loss: 0.1517 - acc: 0.9400 - val_loss: 0.2777 - val_acc: 0.9190
Epoch 4/20
100/100 [==============================] - 445s 4s/step - loss: 0.1588 - acc: 0.9375 - val_loss: 0.2444 - val_acc: 0.9210
Epoch 5/20
100/100 [==============================] - 446s 4s/step - loss: 0.1500 - acc: 0.9380 - val_loss: 0.2139 - val_acc: 0.9240
Epoch 6/20
100/100 [==============================] - 445s 4s/step - loss: 0.1273 - acc: 0.9500 - val_loss: 0.1788 - val_acc: 0.9350
Epoch 7/20
100/100 [==============================] - 445s 4s/step - loss: 0.1316 - acc: 0.9460 - val_loss: 0.1801 - val_acc: 0.9280
Epoch 8/20
100/100 [==============================] - 446s 4s/step - 

In [57]:
import matplotlib.pyplot as plt

In [85]:
# acc=history.history['acc']
# val_acc=history.history['val_acc']
# loss=history.history['loss']
# val_loss=history.history['val_loss']

# plot_epochs=range(1,len(acc)+1)
# plt.plot(plot_epochs,acc,'bo',label='acc')
# plt.plot(plot_epochs,val_acc,'b',label='validation acc')
# plt.legend()
# # # plt.show()


In [93]:
def smooth_curve(points,factor=0.8):
    smooth_point=[]
    for point in points:
        if smooth_point:
            previous=smooth_point[-1]
            smooth_point.append(previous*factor + (1-factor)* point)
        else:
            smooth_point.append(point)
    return smooth_point

In [95]:
# plt.plot(plot_epochs,smooth_curve(acc),'bo',label='acc')
# plt.plot(plot_epochs,smooth_curve(val_acc),'b',label='validation acc')
# plt.legend()